# Tecnología aplicada a la música de tradición oral (bloque 3)
### Demostración 3. Exploración “en vivo” de un jupyter notebook

Partes del flujo de trabajo

1. Cargar una melodía de libre elección.
2. Visualizar la estructura de datos subyacente y renderizarla
3. Filtrar notas, duraciones e intervalos
4. Procesos de extracción de información analítica (*feature extraction*):
> * Histograma de intervalos
> * Histograma de alturas
> * Histograma de duraciones
> * Armadura
> * Rango
> * Altura inicial y final
> * Altura mediana
> * Altura más repetida
> * Altura menos repetida
5. Visualización gráfica:
> * Densidad de notas por compás
> * Progress bar de alturas

In [ ]:
### este código debe ejecutarse obligatoriamente para que el resto funcione. 
## Su cometido es cargar el software que nutre este entorno informático.
from music21 import *
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from collections import Counter
from matplotlib import pyplot as plt
import pandas as pd
from melodic import ambito, armadura, get_counter_stats, rec_notas, note_rest_ratio
from visualization import activityPlot, activityHeatMap, melodicContour

### 1. Cargar una melodía de libre elección

In [ ]:
valor = widgets.ToggleButtons(
    options=['40', '60'],
    description='Seleccionar repertorio según una década:',
    value='40'  # Valor por defecto
)

carpeta = widgets.Label(value=f'Seleccionado: {valor.value}')

def actualizar_carpeta(*args):
    carpeta.value = f'Seleccionado: {valor.value}'

valor.observe(actualizar_carpeta, 'value')

display(valor, carpeta)


In [ ]:
## IMPRIMIR LA LISTA DE ARCHIVOS DE DICHA CARPETA
folder = str(carpeta.value).split()[-1]
mels = os.listdir(folder + '/')

for i in mels:
    print(i)
    

In [ ]:
### MODIFICA EL NOMBRE DE ARCHIVO DE ESTE CÓDIGO Y EJECÚTALO CON PLAY O SHIFT + ENTER

try:
    s = converter.parse(folder + '/' + 'GR_Cuna_M22-018.krn')
    print('La partitura se ha cargado correctamente')
except:
    print('Glups! algo debe haber ido mal. Revisa el nombre del archivo.')

### 2. Visualizar la estructura de datos subyacente y renderizar la partitura

In [ ]:
### MOSTRAR LOS DATOS SUBYACENTES QUE GENERA EL PROGRAMA

s.show('text')

In [ ]:
### VER LA PARTITURA RENDERIZADA

s.show()

### 3. Filtrar notas, duraciones e intervalos

In [ ]:
### AQUÍ CREAMOS FILTROS PARA NOTAS

pitches = s.recurse().notes

## DURACIONES
durs = [i.quarterLength if isinstance(i.quarterLength, float) else round(float(i.quarterLength), 2) for i in pitches]

## E INTERVALOS

ints = list()
lastpitch = None
for i in pitches:
    if isinstance(i, note.Note) == True:
        if lastpitch is not None:
            now = interval.Interval(lastpitch, i).semitones
            ints.append(now)
        else:
            lastpitch = i
    else:
        pass

In [ ]:
### IMPRIMIMOS LOS RESULTADOS DE ESTOS FILTROS

print('Secuencia de alturas: ')
alturas = [i.nameWithOctave for i in pitches]
print(alturas)
print('----')

print('Secuencia de duraciones (unidad de negra): ')
print(durs)

print('----')
print('Secuencia de intervalos (unidad de semitono): ')
print(ints)


In [ ]:
### CONTADORES DE OCURRENCIAS

pitches_counter = Counter(alturas)

durs_counter = Counter(durs)

ints_counter = Counter(ints)

def generateNumericalHistogram(cnt, tag):
    etiquetas = list(cnt.keys())
    valores = list(cnt.values())

    bar_width = 0.8

    plt.bar(etiquetas, valores, color='skyblue', edgecolor='black', width=bar_width)
    plt.xlabel("Tipos")
    plt.ylabel("Frecuencia")
    plt.title("Histograma de " + tag)

    if all(isinstance(e, (int, float)) for e in etiquetas):
        if any(isinstance(e, float) for e in etiquetas):
            step = 1
            plt.xticks(etiquetas, rotation=45, ha='right')
        else:
            plt.xticks(range(int(min(etiquetas)), int(max(etiquetas)) + 1, 1), rotation=45, ha='right')

    plt.tight_layout()
    plt.show()

def generateCategoricalHistogram(cnt, tag):
    etiquetas = sorted(cnt.keys())  # Ordenar etiquetas
    valores = [cnt[e] for e in etiquetas]  # Obtener valores en el mismo orden

    etiquetas_str = [str(e) for e in etiquetas]  # Convertir etiquetas a strings

    bar_width = 0.8

    plt.bar(etiquetas_str, valores, color='skyblue', edgecolor='black', width=bar_width)
    plt.xlabel("Tipos")
    plt.ylabel("Frecuencia")
    plt.title("Histograma de " + tag)

    plt.xticks(etiquetas_str, rotation=45, ha='right')  # Tratar etiquetas como categóricas

    plt.tight_layout()
    plt.show()

In [ ]:
### HISTOGRAMAS DE FRECUENCIAS

In [ ]:
## ¿Cómo se guardan estos conteos en Python?

print(durs_counter)

In [ ]:
generateNumericalHistogram(pitches_counter, 'alturas')

In [ ]:
generateCategoricalHistogram(durs_counter, 'duraciones')

In [ ]:
generateNumericalHistogram(ints_counter, 'intervalos')

### 4. Procesos de extracción de información analítica (*feature extraction*)

#### Obtener la armadura

In [ ]:
ks = armadura(s)
print('La armadura de esta pieza es ' + str(ks))

#### Obtener el rango melódico

In [ ]:
amb = ambito(s)
amb_2 = amb.split(' - ')
amb_3 = str(interval.Interval(note.Note(amb_2[0]), note.Note(amb_2[1])).semitones)
print('El ambito de esta pieza es de ' + amb_3 + f' semitonos ({amb})')

#### Obtener la altura inicial y final

In [ ]:
first_pitch = alturas[0]
last_pitch = alturas[-1]

print('La melodía comienza por ' + first_pitch + ' y acaba por ' + last_pitch)

#### Obtener la altura más repetida

In [ ]:
#### Obtener la altura más repetida
tone_stats = get_counter_stats(rec_notas(s, last_pitch), 'notes')
altura_mas_comun = tone_stats[0] ## más repetido
print('La altura modal es ' + altura_mas_comun)

#### Obtener la altura mediana

In [ ]:
altura_mediana = tone_stats[1] ## mediano repetido
print('La altura mediana es ' + altura_mediana)

#### Obtener la altura menos repetida

In [ ]:
altura_menos_comun = tone_stats[2] ## menos repetido
print('La altura menos repetida es ' + altura_menos_comun)

#### Ratio entre duracion de notas y duración de silencios

In [ ]:
ratio_nt_rst = round(note_rest_ratio(s), 2)
print('El ratio entre la duración de notas y de silencios es de ' + str(ratio_nt_rst))

### 5. Visualización gráfica

#### Gráfico basado en la densidad de notas por compás

In [ ]:
activityPlot(s)

#### Mapa de calor basado en la densidad de notas por compás

In [ ]:
activityHeatMap(s)

#### Gráfico de barras basado en la altura

In [ ]:
melodicContour(pitches)